##### Imports

In [4]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F
from torch.utils.data import random_split
import random
from torchvision.transforms import ToPILImage
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from torch.utils.tensorboard import SummaryWriter

### Tesnorboard

In [5]:
root_path = os.getcwd()

In [6]:
log_dir = os.path.join(root_path, "logs")
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

##### Dataset class (with preprocessed)

In [7]:
class PreprocessedMushroomDataset(Dataset):
    def __init__(self, csv_file, root_dir, has_labels=True):
        self.annotations = pd.read_csv(csv_file, dtype={0: str})
        self.root_dir = root_dir
        self.has_labels = has_labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0] + '.pt')
        image = torch.load(img_name)  
        if self.has_labels:
            label = int(self.annotations.iloc[idx, 1])
        else:
            label = -1 
        return image, label

##### Define paths

In [8]:
root_path = os.path.dirname(os.getcwd())
models_path =  os.path.join(root_path, 'models')
dataset_path = os.path.join(root_path, 'dataset')

dataset_preprocessed_path = os.path.join(dataset_path, 'preprocessed')
preprocessed_train_path = os.path.join(dataset_preprocessed_path, 'train')
preprocessed_test_path = os.path.join(dataset_preprocessed_path, 'test')

csv_path = os.path.join(dataset_path, 'csv_mappings')
train_csv_path = os.path.join(csv_path, 'train.csv')
test_csv_path = os.path.join(csv_path, 'test.csv')


##### Load datasets

In [9]:
train_dataset = PreprocessedMushroomDataset(csv_file=train_csv_path, root_dir=preprocessed_train_path, has_labels=True)
test_dataset = PreprocessedMushroomDataset(csv_file=test_csv_path, root_dir=preprocessed_test_path, has_labels=False)

##### Split dataset into training and validation

In [10]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])


##### Define Dataloaders

In [11]:
train_dataloader = DataLoader(train_subset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_subset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

##### Load model

In [ ]:
model = models.alexnet(pretrained=False)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(train_dataset.annotations['Mushroom'].unique()))

In [ ]:
print(model)

In [14]:
class MushroomNet(nn.Module):
    def __init__(self, num_classes):
        super(MushroomNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = 10 
model = MushroomNet(num_classes).to(device)

##### Params

In [16]:
model = model.to(device)

lr = 0.0001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

num_epochs = 20

##### Train model

In [ ]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Training
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    train_loss = running_train_loss / len(train_dataloader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    
    # Validation
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_loss = running_val_loss / len(val_dataloader)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    # Log to TensorBoard
    writer.add_scalars("Loss", {"Train": train_loss, "Validation": val_loss}, epoch)
    writer.add_scalars("Accuracy", {"Train": train_accuracy, "Validation": val_accuracy}, epoch)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')


In [15]:
writer.close()

##### Loss and accuracy

In [ ]:
# Plot Loss and Accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title("Loss vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title("Accuracy vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

##### Save trained model

In [ ]:
model_save_path = os.path.join(models_path, 'alexnet_model.pth')

In [ ]:
torch.save(model.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

In [ ]:
model = models.alexnet(pretrained=False)
model.load_state_dict(torch.load(model_save_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

##### Test predictions

In [ ]:
model.eval()
test_predictions = []

with torch.no_grad():
    for idx, (images, labels) in enumerate(test_dataloader):
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        for i in range(images.size(0)):
            img_name = test_dataset.annotations.iloc[idx * test_dataloader.batch_size + i, 0]
            test_predictions.append((img_name, predicted[i].item()))

for img_name, pred in test_predictions:
    print(f'Image: {img_name}, Predicted Label: {pred}')


### Show prediction samples 

In [19]:
def denormalize(image, mean, std):
    image = image.clone()
    for t, m, s in zip(image, mean, std):
        t.mul_(s).add_(m)
    return image

In [ ]:
def show_sample_predictions(dataset, predictions, num_samples=10):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    indices = random.sample(range(len(predictions)), num_samples)
    sample_predictions = [predictions[i] for i in indices]

    fig, axes = plt.subplots(1, num_samples, figsize=(20, 4))
    for ax, (img_name, pred) in zip(axes, sample_predictions):
        img_path = os.path.join(preprocessed_test_path, img_name + '.pt')
        image = torch.load(img_path).cpu()
        image = denormalize(image, mean, std)
        image = ToPILImage()(image)
        
        ax.imshow(image)
        ax.set_title(f'Predicted: {pred}')
        ax.axis('off')
    plt.show()

show_sample_predictions(test_dataset, test_predictions, num_samples=10)


##### Class metrics

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def calculate_metrics_per_class(y_true, y_pred, class_names):
    # Classification Report
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
    
    # Global Metrics
    print("Global Metrics:")
    print(f"  Accuracy: {report['accuracy']:.2f}")
    print(f"  Macro Avg Precision: {report['macro avg']['precision']:.2f}")
    print(f"  Macro Avg Recall: {report['macro avg']['recall']:.2f}")
    print(f"  Macro Avg F1-Score: {report['macro avg']['f1-score']:.2f}")
    print()
    
    # Per-Class Metrics
    print("Per-Class Metrics:")
    print(f"{'Class':<15}{'Precision':<10}{'Recall':<10}{'F1-Score':<10}{'Accuracy':<10}")
    print("-" * 55)
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    total_samples_per_class = cm.sum(axis=1) 
    correct_preds_per_class = np.diag(cm)    
    per_class_accuracy = correct_preds_per_class / total_samples_per_class
    
    for idx, class_name in enumerate(class_names):
        metrics = report[class_name]
        accuracy = per_class_accuracy[idx] if total_samples_per_class[idx] > 0 else 0.0
        print(f"{class_name:<15}{metrics['precision']:<10.2f}{metrics['recall']:<10.2f}{metrics['f1-score']:<10.2f}{accuracy:<10.2f}")
    
    print("\nConfusion Matrix:")
    print(cm)


In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

class_names = train_dataset.annotations['Mushroom'].unique().tolist()
calculate_metrics_per_class(all_labels, all_preds, class_names)